# Surface Normal Model Utility functions

In [1]:
import sys
sys.path.append('./models/')
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time
import os

## Test of Loss function for surface normal vectors

In [3]:
# x = torch.autograd.Variable(torch.randn(3, 5) )
# y = torch.autograd.Variable(torch.randn(3, 5) )
x = torch.autograd.Variable(torch.randn(2, 3, 5, 5) )
y = torch.autograd.Variable(torch.randn(2, 3, 5, 5) )
print('Orig vector (only first one printed):')
print(x)
target2 = torch.autograd.Variable(torch.tensor([1], dtype=torch.float))

#CosineEmbeddingLoss
loss_fn = torch.nn.CosineEmbeddingLoss(reduction='none') #NOTE: elementwise_mean apparently doesn't work
z0 = loss_fn(x, y, target2)
print('\nCosineEmbeddingLoss')
print(z0)

#CosineSimilarity
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
loss_fn = 1 - cos(x,y)
print('\nCosine Similarity Matrix')
print(loss_fn)
loss_fn_mean = torch.mean( loss_fn )
print('Cosine Similarity mean loss')
print(loss_fn_mean)
        

#Custom Code
# a = (x.norm(p=2,dim=0)*y.norm(p=2,dim=0))
# b = torch.sum(x*y,1)
# a[a==0] = 1
# b[a==0] = 1
# distance = 1 - ( b / a)
# totloss = torch.mean(distance)
# print('\nShuran\'s Loss func')
# print(totloss)


Orig vector (only first one printed):
tensor([[[[-0.8371,  1.5100,  2.0602, -0.6620,  0.8571],
          [ 0.4888, -1.0918,  1.6563,  0.9894, -0.0242],
          [-0.0954, -0.0485,  0.7650, -0.9491, -0.0061],
          [-0.1170, -0.7213,  0.6771,  1.0299,  0.4900],
          [ 0.6074, -0.4707,  1.2304,  0.5665, -2.0613]],

         [[-0.6722,  0.8400,  0.5930, -1.2605, -0.5393],
          [-0.3872,  0.4582,  0.1225,  0.5573, -0.1200],
          [ 1.0336,  0.8583, -1.7595,  0.4496, -0.6732],
          [ 0.8210,  0.5038,  0.1046, -0.8631,  0.3233],
          [ 0.0631,  0.0158, -0.5698, -0.0013, -0.8978]],

         [[-0.1006,  0.1529,  0.7249, -0.1935,  0.0963],
          [ 0.0705,  0.8292,  1.3882,  1.0618, -0.8904],
          [-0.3433,  0.3626,  0.1260, -0.3518, -2.6571],
          [ 2.2381,  0.7644, -0.2176,  0.3785, -0.2013],
          [ 1.1475,  0.0581, -0.5470,  0.0702,  0.5270]]],


        [[[-1.4632, -0.4606,  1.0216, -0.3054,  0.6981],
          [-1.3388, -1.1526, -0.2935, -1.6

## Test of Normalization Function for surface normal vectors

In [16]:
print('Loss matrix dimensions:')
print(loss_fn.size())
print('\nNormalized Tensor, along dim 1 (column):')
print(nn.functional.normalize(loss_fn, p=2, dim=0))

# k = torch.tensor([[0.5, 0.1],[0.2,0.5]], dtype=torch.float)
# print('Tensor k:')
# print(k)
# print('\nNormalized Tensor, along dim 1 (column):')
# print(nn.functional.normalize(k, p=2, dim=1))

Loss matrix dimensions:
torch.Size([2, 5, 5])

Normalized Tensor, along dim 1 (column):
tensor([[[0.6780, 0.7880, 0.5233, 0.9966, 0.4701],
         [0.8541, 0.8137, 0.9492, 0.0585, 0.7046],
         [0.9569, 0.8895, 0.6367, 0.8311, 0.4086],
         [0.6550, 0.8086, 0.3456, 0.8344, 0.8255],
         [0.9886, 0.1599, 0.9286, 0.9528, 0.9968]],

        [[0.7351, 0.6157, 0.8522, 0.0827, 0.8826],
         [0.5200, 0.5812, 0.3146, 0.9983, 0.7096],
         [0.2904, 0.4570, 0.7711, 0.5562, 0.9127],
         [0.7556, 0.5884, 0.9384, 0.5511, 0.5645],
         [0.1503, 0.9871, 0.3711, 0.3036, 0.0794]]])


## Read EXR Files

### OpenEXR - Single Channel
This method uses the OpenEXR library to read in the EXR files. How to load in single channel (depth):

In [4]:
import OpenEXR, Imath
EXR_PATH = '/media/shrek/work/datasets/greppy-surface-normals-bottle/normals_masks/000000000-normals.exr'
exr_file = OpenEXR.InputFile(EXR_PATH)

cm_dw = exr_file.header()['dataWindow']
render_depth = np.frombuffer(
    exr_file.channel('R', Imath.PixelType(Imath.PixelType.HALF)),
    dtype=np.float16
)

render_depth.shape = (cm_dw.max.y - cm_dw.min.y + 1, cm_dw.max.x - cm_dw.min.x + 1) # rows, cols

print(cm_dw)
print(render_depth.shape)

(0, 0) - (1919, 1079)
(1080, 1920)


### OpenEXR - 3 Channels
How to load in 3 channels (surface normal):

In [5]:
def exr_loader(path, ndim=3):
    """
    loads an .exr file as a numpy array
    :param path: path to the file
    :param ndim: number of channels that the image has,
                    if 1 the 'R' channel is taken
                    if 3 the 'R', 'G' and 'B' channels are taken
    :return: np.array containing the .exr image
    """

    exr_file = OpenEXR.InputFile(EXR_PATH)
    cm_dw = exr_file.header()['dataWindow']
    size = (cm_dw.max.x - cm_dw.min.x + 1, cm_dw.max.y - cm_dw.min.y + 1)

    pt = Imath.PixelType(Imath.PixelType.FLOAT)

    if ndim == 3:
        # read channels indivudally
        allchannels = []
        for c in ['R', 'G', 'B']:
            # transform data to numpy
            channel = np.frombuffer(exr_file.channel(c, pt), dtype=np.float32)
            channel.shape = (size[1], size[0])
            allchannels.append(channel)

        # create array and transpose dimensions to match numpy style
        exr_arr = np.array(allchannels).transpose((0, 1, 2)) 
        return exr_arr
    
    if ndim == 1:
        # transform data to numpy
        channel = np.fromstring(channel=pic.channel('R', pt), dtype=np.float32)
        channel.shape = (size[1], size[0])  # Numpy arrays are (row, col)
        exr_arr = np.array(channel)
        return exr_arr

im = exr_loader(EXR_PATH, ndim=3)
print(im.shape)

(3, 1080, 1920)


### Pre-Processing EXR numpy array
It will output a tensor which is scaled to a given size and converted to a tensor.

In [7]:
import torchvision
from torchvision import transforms, utils
from PIL import Image

imsize = 224

def transformLabel(label):
        transform_list = []
        transform_list.append(transforms.Resize([imsize, imsize]))
        transform_list.append(transforms.ToTensor())
        tf = transforms.Compose(transform_list)
        label = tf(label)
        return label

label = torch.from_numpy(im)
label = transforms.ToPILImage()(label, 'F')
label = transformLabel(label)
# print(label.type)
label

TypeError: __call__() takes 2 positional arguments but 3 were given

In [1]:
import pathlib